## Bell Study Model Analysis

This is the preliminary Bell Study analysis NN model using Keras, AutoKeras, KerasTuner. 

> Author : Y.Y. Gary Peng  
> Istitute: National Tsing Hua University, Taiwan and ATLAS experiment, CERN  
> License : MIT  
> Data : August 19, 2023  
> Email : yuan-yen.peng@cern.ch

Version : 1.0


In [1]:
import os
import gc
import warnings
import numpy as np
import pandas as pd
from scipy import stats
from numba import njit
import mplhep as hep
import matplotlib.pyplot as plt
import plot.AtlasPlot as atlas
import glob
import uproot
import ROOT

Welcome to JupyROOT 6.28/04


In [2]:
def get_files_names(path):
    files_name = glob.glob(path)
    return files_name

path = "./truth/*/*.npz"
files_name = get_files_names(path)
print(files_name)

def get_data(path):
    try:
        with np.load(path, allow_pickle=True) as f:
            data_dict = {name: f[name] for name in f.files}
            return pd.DataFrame(data_dict)
    except FileNotFoundError:
        print("File not found!")
        return pd.DataFrame()
    
files = []
files_name.sort()
for f in files_name:
    files.append(get_data(f))
    
# need to check the order of data name.
files_name

# need to follow the order of data name.
(
    CGLMP,
    Higgs,
    LeadLep,
    LepM,
    LepP,
    LmNu,
    LpNu,
    MET,
    Wm,
    Wp,
    diLep,
    SubLep,
    Xi,
) = files
GEV=1e3

del files_name, files
gc.collect()

['./truth/archive/CGLMP.npz', './truth/archive/WpBoson.npz', './truth/archive/subLeadLepton.npz', './truth/archive/LeptonP.npz', './truth/archive/Higgs.npz', './truth/archive/LeadLepton.npz', './truth/archive/LeptonM.npz', './truth/archive/xi_xyz.npz', './truth/archive/WmBoson.npz', './truth/archive/LmNu.npz', './truth/archive/LpNu.npz', './truth/archive/MET.npz', './truth/archive/diLepton.npz']


422

In [3]:
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

NotFoundError: dlopen(/opt/homebrew/Caskroom/miniconda/base/envs/cern/lib/python3.10/site-packages/tensorflow-plugins/libmetal_plugin.dylib, 0x0006): symbol not found in flat namespace '__ZN10tensorflow8internal10LogMessage16VmoduleActivatedEPKci'

In [ ]:
model = keras.models.load_model('./models/KerasTuner.h5')